# Auto Batching

<!-- ![](auto-batch.png) -->
<img src="auto-batch.png" alt="drawing" style="width:70%;"/>

## Bacthed Queue: `.get()` a batched output 
### Single Consumer - Multiple Publisher

`get multiple items not more than the specified batch size` **and** `get the items within the timeout`

In [ ]:
from queue import Queue, Empty
import time
from threading import Thread
from threading import Event
from typing import Callable, List
import random
import uuid

from typing import Any
from dataclasses import dataclass, field
import uuid
from threading import Event

In [ ]:
class BatchedQueue:
    def __init__(self, timeout=1.0, bs=1):
        self.timeout = timeout
        self.bs = bs
        self._queue: Queue = Queue()
        self._result = []
        self._event = Event()

    def get(self):
        entered_at = time.time()
        timeout = self.timeout
        bs = self.bs

        if self._queue.qsize() >= bs:
            return [self._queue.get_nowait() for _ in range(bs)]

        while (
            self._event.wait(timeout - (time.time() - entered_at))
            and self._queue.qsize() < bs
        ):
            True

        result = []
        try:
            for _ in range(bs):
                result.append(self._queue.get_nowait())
            return result
        except Empty:
            return result

    def put(self, item):
        self._queue.put(item)
        if self._event.is_set() and self.size >= self.bs:
            self._event.set()

    @property
    def size(self):
        return self._queue.qsize()

In [ ]:
q = BatchedQueue(timeout=2, bs=4)

q.put(1)
q.size

In [ ]:
t0 = time.time()
print("size", q.size)
q.get()
print("size", q.size)
time.time() - t0

### Test with a publisher

In [ ]:
import random

In [ ]:
q = BatchedQueue(timeout=2, bs=4)


def publisher():
    for i in range(16):
        time.sleep(random.randint(0, 1))
        q.put(random.randint(1000, 100000))


thread1 = Thread(target=publisher, daemon=True)
thread2 = Thread(target=publisher, daemon=True)
thread3 = Thread(target=publisher, daemon=True)

thread1.start()
thread2.start()
thread3.start()

In [ ]:
q.size

In [ ]:
for i in range(12):
    t0 = time.time()
    items = q.get()
    print(items)
    t1 = time.time()
    print(f"consumed in {t1-t0:.2f}")

## Batched Processor

In [ ]:
@dataclass
class WaitedObject:
    item: Any = None
    result: Any = None
    _event: Event = None
    created_at: float = field(default_factory=lambda: time.time())
    completed_at: float = None

    def __post_init__(self):
        self._event = Event()

    def set_result(self, result) -> None:
        self.result = result
        self.completed_at = time.time()
        self._event.set()

    @property
    def completed(self) -> bool:
        return self._event.is_set()

    @property
    def completion_time(self) -> str:
        if self.completed_at:
            return f"{self.completed_at - self.created_at:.3f}s"
        else:
            return "Waiting"

    def get(self, timeout: float = None) -> Any:
        if self.completed:
            return self.result
        self._event.wait(timeout)
        return self.result

    def __repr__(self) -> str:
        return f"WaitedOjb({dict(item=self.item, completed=self.completed, result=self.result, completion_time=self.completion_time)})"

In [ ]:
a = WaitedObject(item=1)
a.set_result(2)
a.get(1)

In [ ]:
class BatchProcessor:
    def __init__(
        self,
        func: Callable,
        timeout=4.0,
        bs=1,
    ):
        self._batched_queue = BatchedQueue(timeout=timeout, bs=bs)
        self.func = func
        self._event = Event()
        self._cancel_signal = Event()

        self._thread = Thread(target=self._process_queue)
        self._thread.start()

    def _process_queue(self):
        print("Started processing")
        while True:
            if self._cancel_signal.is_set():
                print("Stopped batch processor")
                return
            t0 = time.time()
            batch: List[WaitedObject] = self._batched_queue.get()
            t1 = time.time()
            # print(f"waited {t1-t0:.2f}s for batch")
            if not batch:
                # print("no batch")
                continue
            batch_items = [b.item for b in batch]
            # print(batch_items)
            results = self.func(batch_items)
            for b, result in zip(batch, results):
                b.set_result(result)

    def process(self, item: Any):
        waited_obj = WaitedObject(item=item)
        self._batched_queue.put(waited_obj)
        return waited_obj

    def cancel(self):
        self._cancel_signal.set()
        self._thread.join()

In [ ]:
def fake_ml_api(X):
    n = len(X)
    # print(f"{n} items")
    for i in range(1000):
        for j in range(1000):
            for k in range(50):
                l = i / 1000 * j / 1000
    return [x + l for x in X]

In [ ]:
processor = BatchProcessor(fake_ml_api, timeout=1, bs=4)

In [ ]:
results = []
for i in range(8):
    x = processor.process(i)
    results.append(x)

In [ ]:
results

In [ ]:
print(results[2].get())
print(results[7].get())

In [ ]:
results